# tweaking pandas slicing as a binary search O(log(n))

In [1]:
import copy
import csv
import pandas as pd
import numpy as np
import scipy as scipy
from scipy.stats import pearsonr
import math
import matplotlib
import matplotlib.pyplot as plt
#from matplotlib_venn import venn3, venn3_circles, venn2, venn2_circles
import itertools
from copy import deepcopy
from collections import defaultdict
from datetime import datetime
import random
pd.options.display.mpl_style = 'default'
%matplotlib inline
import pickle
#%pylab inline
import os
import matplotlib.ticker as mtick
from scipy.stats import chi2_contingency
import re

/home/oryoruk/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: 
mpl_style had been deprecated and will be removed in a future version.
Use `matplotlib.pyplot.style.use` instead.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
def check_sub(snps,chrom,chrom_start,reverse_strand):
    snp_at_site = snps[(snps.chrom==chrom)&(snps.chrom_start==chrom_start)]
    if snp_at_site.empty:
        return (False, '')
    #if there is a snp:
    else:
        #return sub=True and the alt nucleotide
        #take the reverse complement if working on the reverse strand
        if reverse_strand:
            return (True, reverse_complement(snp_at_site.iloc[0].alt) )
        else:
            return (True, snp_at_site.iloc[0].alt )

In [3]:
def reverse_complement(seq):
    complement_code = dict( zip( "ATCGNatcgn" , "TAGCNtagcn" ) )
    #return "".join( complement_code[nuc] for nuc in reversed(seq) )
    return "".join( complement_code[nuc] for nuc in seq[::-1] )

In [4]:
def check_sub(snps,chrom,chrom_start,reverse_strand):
    snp_at_site = snps[(snps.chrom==chrom)&(snps.chrom_start==chrom_start)]
    if snp_at_site.empty:
        return (False, '')
    #if there is a snp:
    else:
        #return sub=True and the alt nucleotide
        #take the reverse complement if working on the reverse strand
        if reverse_strand:
            return (True, reverse_complement(snp_at_site.iloc[0].alt) )
        else:
            return (True, snp_at_site.iloc[0].alt )

In [5]:
def check_sub_exists(snps,chrom,chrom_start):
    snp_at_site = snps[(snps.chrom==chrom)&(snps.chrom_start==chrom_start)]
    if snp_at_site.empty:
        return False
    #if there is a snp:
    else:
        return True

In [6]:
snp_file_name = 'AFR_SNPs_only_no_singletons.bed'

In [7]:
chrom = '1'

In [109]:
snp_file = './../../data/mutation_datasets/snp/1kg_phaseIII/' + snp_file_name

In [110]:
# read in the file that contains all the substitutions (only a slice of it that is needed)
snps = pd.read_csv(snp_file, delimiter='\t', header=None,
                          #names=['chrom', 'chrom_start', 'ref', 'alt', 'alt2', 'id', 'freq'],
                        #changed this line to accomodate phase III data files I created:
                        names=['chrom', 'chrom_start', 'ref', 'alt'],
                   dtype={'chrom': object})

In [111]:
snps = snps[snps.chrom == chrom]

In [112]:
snps.head()

,chrom,chrom_start,ref,alt
0,1,10505,A,T
1,1,10506,C,G
2,1,10579,C,A
3,1,10642,G,A
4,1,11008,C,G


In [113]:
snps.tail()

,chrom,chrom_start,ref,alt
3078485,1,249239747,G,C
3078486,1,249239762,G,T
3078487,1,249239902,G,T
3078488,1,249240219,A,T
3078489,1,249240539,T,G


In [13]:
%%time
(snps['chrom_start'] == 249239902).any()

CPU times: user 55 ms, sys: 0 ns, total: 55 ms
Wall time: 53.8 ms


True

In [14]:
%%time
snps[(snps.chrom==chrom)&(snps.chrom_start==249239902)].empty

CPU times: user 338 ms, sys: 4 ms, total: 342 ms
Wall time: 341 ms


False

# .any() is 6 times faster than slicing

In [15]:
snps.head()

,chrom,chrom_start,ref,alt
0,1,10505,A,T
1,1,10506,C,G
2,1,10579,C,A
3,1,10642,G,A
4,1,11008,C,G


In [19]:
snps.tail()

,chrom,chrom_start,ref,alt
3078485,1,249239747,G,C
3078486,1,249239762,G,T
3078487,1,249239902,G,T
3078488,1,249240219,A,T
3078489,1,249240539,T,G


In [17]:
snps.chrom_start.searchsorted(249239902,'left')

array([3078487])

In [20]:
%%time
snps.chrom_start.searchsorted(249239902,'left')

CPU times: user 1 ms, sys: 0 ns, total: 1 ms
Wall time: 360 µs


array([3078487])

In [18]:
len(snps)

3078490

# searchsorted() is ~x947 times faster

In [43]:
chrom_start = 249240539
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

True

In [44]:
chrom_start = 249240538
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

False

In [45]:
chrom_start = 249240512
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

False

In [46]:
chrom_start = 249240219
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

True

In [47]:
chrom_start = 249240218
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

False

In [48]:
%%time
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 440 µs


False

In [51]:
%%timeit
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

1000 loops, best of 3: 236 µs per loop


# finally let's try numpy's bisect for binary search

In [57]:
snps_chrom_start_list = list(snps.chrom_start)

In [58]:
len(snps_chrom_start_list)

3078490

In [59]:
all(snps_chrom_start_list[i] <= snps_chrom_start_list[i+1] for i in xrange(len(snps_chrom_start_list)-1))

True

In [60]:
from bisect import bisect_left

def binary_search(a, x, lo=0, hi=None):  # can't use a to specify default for hi
    hi = hi if hi is not None else len(a)  # hi defaults to len(a)   
    pos = bisect_left(a, x, lo, hi)  # find insertion position
    return (pos if pos != hi and a[pos] == x else -1)  # don't walk off the end

In [62]:
chrom_start = 249240218
binary_search(snps_chrom_start_list,chrom_start)

-1

In [63]:
chrom_start = 249240219
binary_search(snps_chrom_start_list,chrom_start)

3078488

In [64]:
%%timeit
chrom_start = 249240219
binary_search(snps_chrom_start_list,chrom_start)

The slowest run took 4.20 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 3.8 µs per loop


# binary search is the winner

# all versions of the same operation

In [96]:
len(snps)

3078490

In [97]:
snps.iloc[2*10**6+1]

chrom                  1
chrom_start    171455667
ref                    A
alt                    G
Name: 2000001, dtype: object

### pandas slicing

In [98]:
chrom_start = 171455621
print not snps[(snps.chrom==chrom)&(snps.chrom_start==chrom_start)].empty
chrom_start = 171455622
print not snps[(snps.chrom==chrom)&(snps.chrom_start==chrom_start)].empty

True
False


In [99]:
%%timeit
chrom_start = 171455621
not snps[(snps.chrom==chrom)&(snps.chrom_start==chrom_start)].empty
chrom_start = 171455622
not snps[(snps.chrom==chrom)&(snps.chrom_start==chrom_start)].empty

1 loop, best of 3: 659 ms per loop


### pandas any() method

In [100]:
chrom_start = 171455621
print (snps['chrom_start'] == chrom_start).any()
chrom_start = 171455622
print (snps['chrom_start'] == chrom_start).any()

True
False


In [101]:
%%timeit
chrom_start = 171455621
(snps['chrom_start'] == chrom_start).any()
chrom_start = 171455622
(snps['chrom_start'] == chrom_start).any()

10 loops, best of 3: 107 ms per loop


### pandas searchsorted() method (using numpy.searchsorted() which is a heavy version of simple binary search copying the entire data if non-contigous)

In [102]:
chrom_start = 171455621
print snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start
chrom_start = 171455622
print snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

True
False


In [103]:
%%timeit
chrom_start = 171455621
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start
chrom_start = 171455622
snps.chrom_start.iloc[ snps.chrom_start.searchsorted(chrom_start,'left')[0] ] == chrom_start

1000 loops, best of 3: 472 µs per loop


### list binary search

In [104]:
chrom_start = 171455621
print binary_search(snps_chrom_start_list,chrom_start)
chrom_start = 171455622
print binary_search(snps_chrom_start_list,chrom_start)

2000000
-1


In [105]:
%%timeit
chrom_start = 171455621
binary_search(snps_chrom_start_list,chrom_start)
chrom_start = 171455622
binary_search(snps_chrom_start_list,chrom_start)

100000 loops, best of 3: 8.15 µs per loop


### clear winner is the binary search in this case

In [106]:
chrom_start = 171455621
print binary_search(snps.chrom_start,chrom_start)
chrom_start = 171455622
print binary_search(snps.chrom_start,chrom_start)

2000000
-1


In [118]:
chrom_start = 171455621
sub_index = binary_search(snps.chrom_start,chrom_start)

In [123]:
snps.iloc[sub_index-5:sub_index+5]

,chrom,chrom_start,ref,alt
1999995,1,171455306,T,G
1999996,1,171455322,C,T
1999997,1,171455379,G,A
1999998,1,171455475,A,G
1999999,1,171455502,C,G
2000000,1,171455621,C,A
2000001,1,171455667,A,G
2000002,1,171455792,C,T
2000003,1,171455797,G,A
2000004,1,171455834,G,C


In [122]:
snps.iloc[sub_index]

chrom                  1
chrom_start    171455621
ref                    C
alt                    A
Name: 2000000, dtype: object

In [124]:
snps.alt[sub_index]

'A'